<h2><center>Невязкое уравнение Бюргерса</center></h2>
$$\frac{\partial u}{\partial t} +  u \frac{\partial u}{\partial x} = 0$$

Источник: <br>
https://github.com/urban-fasel/EnsembleSINDy <br>
https://github.com/urban-fasel/EnsembleSINDy/blob/main/PDE-FIND/datasets/burgers.mat <br>


$$256\times256$$
$$ x \in [-4000; 4000]$$
$$ t \in [0; 4]$$
----

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt

import bamt.Networks as Nets
import bamt.Nodes as Nodes
import bamt.Preprocessors as pp
from pgmpy.estimators import K2Score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
None

### 1. Предварительная обработка

In [2]:
df = pd.read_csv('output_main_burgers_equation.csv', index_col = 'Unnamed: 0', sep='\t', encoding='utf-8')
df.shape

(82, 9)

In [3]:
for col in df.columns:
        df[col] = df[col].round(decimals = 10)

In [4]:
# Удаление строк, если все коэф. левой части равны нулю, а коэф. правой части -1 (по сумме элементов строки)
df = df.loc[(df.sum(axis=1) != -1), (df.sum(axis=0) != 0)]
# Удаление нулевых столбцов
df = df.loc[:, (df != 0).any(axis=0)]
# Кол-во ненулевых значений в столбцах/структурах
(df != 0).sum(axis = 0)

du/dx1{power: 1.0}                      66
C                                        3
du/dx2{power: 1.0} * u{power: 1.0}      16
du/dx2{power: 1.0} * u{power: 1.0}_r    66
du/dx1{power: 1.0}_r                    13
du/dx2{power: 1.0}_r                     3
dtype: int64

In [5]:
df.shape

(82, 6)

In [6]:
df_new = df
for col in df_new.columns:
    if '_r' not in col and col + "_r" in df_new.columns: # объединение повторных структур (справа и слева ур-ния)
        temp = df_new[col + "_r"] + df_new[col]
        arr_value = temp.unique()
        arr_value.sort()
        if len(arr_value) == 2 and (arr_value == np.array([-1, 0])).all(): # отделение структур правой части
            df_new[col + "_r"] = df_new[col + "_r"] + df_new[col]
            df_new = df_new.drop(col, axis=1)
        else:
            df_new[col] = df_new[col + "_r"] + df_new[col]
            df_new = df_new.drop(col + "_r", axis=1) 

In [7]:
df_new.shape

(82, 4)

In [9]:
for col in df_new.columns:
    if '_r' in col:
        df_new = df_new.astype({col: "int64"})
        df_new = df_new.astype({col: "str"}) # дискретные значение оборачиваем в строку для корректной работы при обучении

In [10]:
df_new.groupby(df_new.columns.tolist(),as_index=False).size()

,C,du/dx2{power: 1.0} * u{power: 1.0},du/dx1{power: 1.0}_r,du/dx2{power: 1.0}_r,size
0,-0.014616,0.001522,0,-1,3
1,-0.000000,-1.000000,-1,0,79


In [11]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82 entries, 0 to 81
Data columns (total 4 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   C                                   82 non-null     float64
 1   du/dx2{power: 1.0} * u{power: 1.0}  82 non-null     float64
 2   du/dx1{power: 1.0}_r                82 non-null     object 
 3   du/dx2{power: 1.0}_r                82 non-null     object 
dtypes: float64(2), object(2)
memory usage: 3.2+ KB


### 2.Инициализация байесовской сети

In [12]:
discretizer = preprocessing.KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
p = pp.Preprocessor([('discretizer', discretizer)])  # Только дискретизация
data, est = p.apply(df_new)
info_r = p.info

In [13]:
data

,C,du/dx2{power: 1.0} * u{power: 1.0},du/dx1{power: 1.0}_r,du/dx2{power: 1.0}_r
0,0,0,-1,0
1,0,0,-1,0
2,0,0,-1,0
3,0,0,-1,0
4,0,0,-1,0
...,...,...,...,...
77,0,0,-1,0
78,0,0,-1,0
79,0,0,-1,0
80,0,0,-1,0


In [14]:
bn = Nets.HybridBN(has_logit=True, use_mixture=False) # Тип сети (Гибридный - правая часть имеет дискретные значения)

In [15]:
bn.add_nodes(info_r) # Создание узлов

In [16]:
bn.get_info()

,name,node_type,data_type,parents,parents_types
0,C,Gaussian,cont,[],[]
1,du/dx2{power: 1.0} * u{power: 1.0},Gaussian,cont,[],[]
2,du/dx1{power: 1.0}_r,Discrete,disc,[],[]
3,du/dx2{power: 1.0}_r,Discrete,disc,[],[]


In [17]:
# Корневые узлы (структура/ы, которые встречаются наиболее часто)
params = {'init_nodes': ['du/dx2{power: 1.0} * u{power: 1.0}']}

In [18]:
bn.add_edges(data, scoring_function=('K2', K2Score), params=params)

  0%|          | 0/1000000 [00:00<?, ?it/s]

### 3.Обучение параметров сети и семплирование

In [21]:
%%time
bn.fit_parameters(df_new)

CPU times: total: 0 ns
Wall time: 4 ms


In [22]:
def get_objects(synth_data): # without config_bamt
    """
    Parameters
        ----------
        synth_data : pd.dataframe
            The fields in the table are structures of received systems/equations,
            where each record/row contains coefficients at each structure
        config_bamt:  class Config from TEDEouS/config.py contains the initial configuration of the task
    Returns
        -------
        objects_result - list objects (combination of equations or systems)
    """
    objects = [] # equations or systems
    for i in range(len(synth_data)):
        object_row = {}
        for col in synth_data.columns:
            object_row[synth_data[col].name] = synth_data[col].values[i]
        objects.append(object_row)

    objects_result = []
    for i in range(len(synth_data)):
        object_res = {}
        for key, value in objects[i].items():
            if abs(float(value)) > 0.00001: # > config_bamt.params["glob_bamt"]["lambda"]:
                object_res[key] = value
        objects_result.append(object_res)

    return objects_result

In [23]:
objects_res = []
sample_k = 35
while len(objects_res) < sample_k:
    synth_data = bn.sample(50, as_df=True)
    temp_res = get_objects(synth_data)

    if len(temp_res) + len(objects_res) > sample_k:
        objects_res += temp_res[:sample_k - len(objects_res)]
    else:
        objects_res += temp_res

100%|██████████| 50/50 [00:00<00:00, 16611.10it/s]


In [27]:
import math
for col in synth_data.columns:
    column_temp = synth_data[col].astype(float)
    column_main = column_temp[column_temp != 0] # Ноль - это не коэффициент, а параметр, что данная структура отсутствует в ур-нии
    if len(column_main) < 30:
        continue
    print(f'Структура = {col}')    
    print(f'Количество строк: {len(column_main)}')
    m = np.mean(column_main)
    print(f'Выборочное среднее (m) = {m}')
    s = math.sqrt(np.sum(list(map(lambda x: (x - m)**2, column_main)))/(len(column_main) - 1)) # cтандартное отклонение выборки)
    print(f'Cтандартное отклонение выборки (s) = {s}')
    # Доверительный интервал для среднего при неизвестной дисперсии
    maximum_estimation_error = 1.96 * s / math.sqrt(len(column_main))
    print(f'maximum_estimation_error = {maximum_estimation_error}')
    lower = m - maximum_estimation_error
    upper = m + maximum_estimation_error
    print(f'interval estimation: {lower} <= m <= {upper}')
    print('--------------------------')

Структура = C
Количество строк: 50
Выборочное среднее (m) = -0.001593555372866898
Cтандартное отклонение выборки (s) = 0.003140235309248354
maximum_estimation_error = 0.0008704288192228508
interval estimation: -0.002463984192089749 <= m <= -0.0007231265536440471
--------------------------
Структура = du/dx2{power: 1.0} * u{power: 1.0}
Количество строк: 50
Выборочное среднее (m) = -0.9692454314258486
Cтандартное отклонение выборки (s) = 0.19189518250948853
maximum_estimation_error = 0.053190630853159886
interval estimation: -1.0224360622790085 <= m <= -0.9160548005726887
--------------------------
Структура = du/dx1{power: 1.0}_r
Количество строк: 48
Выборочное среднее (m) = -1.0
Cтандартное отклонение выборки (s) = 0.0
maximum_estimation_error = 0.0
interval estimation: -1.0 <= m <= -1.0
--------------------------
